# Webpack详解
## 对脚本和样式的处理
### 1. JS用什么loader加载
**可以使用原生的JavaScript，也可以使用babel.**

因为适配性的问题，所以我们选择原生的JavaScript来进行。
### 2. 多entry
**要使用多个入口文件，可以在webpack.config.js中进行设置。**

In [ ]:
var config = {
    entry : {
        'index' : ['./src/pages/index/index.js'],
        'login' : ['./src/pages/login/index.js'],
    },
    output: {
        path: './dist',
        filename: 'js/[name].js'
    }
};

module.exports = config;

# entry表示多个入口文件，可以使用字典类型来设置

# output中的filename可以来设置路径及按名称来生成js文件

### 3. Jquery引入方法

分为全局的引入方法以及变量的引入方法。

- 基于变量的引入方法：
先使用npm安装jquery,之后在相应的模块中（即js文件中)require一个jquery对象进行引入。

- 全局引入的方法
    - 在html中直接src引入script脚本，之后就可以在各个模块中使用jquery了
    - 有时用户没有在全局安装jquery,想要方便的在webpack工程中引入jquery的依赖，就可以在webpack.config.js中加入一个插件选项

In [ ]:
externals : {
        'jquery': 'window.jQuery'
    }

### 4. 想提取公共模块（工具模块）
> 使用CommonChunkPlugins

在webpack.config.js中，加入plugin选项

In [ ]:
plugins : [
        new webpack.optimize.CommonsChunkPlugin({
            name : 'commons',
            filename:'js/base.js'
        })
    ]
    
# 这段代码表示将模块中共有的require到的元素提取出来打包到js/base.js文件中。

> 总结来说CommonChunkPlugin的作用是将文件中共有的模块自动添加到filename指定的文件中，并且如果在config的entry中有设置Commonchunk选项中name的同名entry（本例中为'commons')则可以向那个文件中写入公共模块（就是每个模块加载前都会使用的模块）有效优化加载速度.

In [ ]:
# webpack.config.js

var webpack = require('webpack')
var config = {
    entry : {
        'common' : ['./src/pages/common/index.js'],
        'index' : ['./src/pages/index/index.js'],
        'login' : ['./src/pages/login/index.js'],
    },
    output: {
        path: './dist',
        filename: 'js/[name].js'
    },
    externals : {
        'jquery': 'window.jQuery'
    },
    plugins : [
        new webpack.optimize.CommonsChunkPlugin({
            name : 'common',
            filename:'js/base.js'
        })
    ]
};

module.exports = config;

# 可以在common指定的entry文件中设置公用的模块，在加载模块时会自动先加载
# 此插件也会探测公用的模块加入js/base.js

### 5. 样式(CSS)使用怎样的loader
> 主要使用的loader主要是**style-loader**与**css-loader**

首先要在npm中安装这两个loader:npm install css-loader style-loader --save-dev

之后要在webpack.config.js中定义要使用的loader:

In [ ]:
module : {
        loaders: [
            { test: /\.css$/, loader: "style-loader!css-loader" }
        ]
    },
    
# 声明这个模块说明当遇到.css文件时，就使用配置的loader
# loader优先级时从右向左，即css-loader之后style-loader

之后就可以在模块js文件中require css文件来为文件导入css

### 6. webpack打包的css怎么独立成单独的文件
> npm install extract-text-webpack-plugin --save-dev

这个插件将css打包到制定的文件中，加快css的加载速度

In [ ]:
# 使用规则有三步

var webpack = require('webpack');
# 1. 先声明插件的变量
var ExtractTextPlugin = require("extract-text-webpack-plugin");

var config = {
    entry : {
        'common' : ['./src/pages/common/index.js'],
        'index' : ['./src/pages/index/index.js'],
        'login' : ['./src/pages/login/index.js'],
    },
    output: {
        path: './dist',
        filename: 'js/[name].js'
    },
    externals : {
        'jquery': 'window.jQuery'
    },
    module : {
        loaders: [
            # 3.在loader中设置插件
            { test: /\.css$/, loader: ExtractTextPlugin.extract("style-loader","css-loader") }
        ]
    },
    plugins : [
        new webpack.optimize.CommonsChunkPlugin({
            name : 'common',
            filename:'js/base.js'
        }),
        # 2. 导入插件，其中要设置打包css的路径及命名方式。
        new ExtractTextPlugin("css/[name].css")
    ]
};

module.exports = config;

### 7. 如何快速生成html文件
生成加载好js模块的html文件，可以用于生成模板以及最终网页的交付。

- 首先安装HtmlWebpackPlugin
> npm install --save-dev html-webpack-plugin
- 之后生成对象HtmlWebpackPlugin
> var HtmlWebpackPlugin = require('html-webpack-plugin');
- 最后在plugin中进行配置，所使用的模板以及最终生成到哪个目标文件中
- 分块模板的实现，有的时候需要制作html中某块的模板，可以使用html-loader来实现。
> 首先要全局安装html-loader
,之后使用模板时<%= require('html-loader!./layout/html-head.html') %>就可以将模板载入

In [ ]:
# webpack.config.js


var webpack = require('webpack');
var ExtractTextPlugin = require("extract-text-webpack-plugin");
var HtmlWebpackPlugin = require('html-webpack-plugin');

//获取htmlWebpackplugin参数
var getHtmlConfig = function(name){
    return {
        template : './src/view/' + name + '.html',
        filename : 'view/' + name + '.html',
        inject : true,
        hash : true,
        chunks : ['common',name]
    }
}

var config = {
    entry : {
        'common' : ['./src/pages/common/index.js'],
        'index' : ['./src/pages/index/index.js'],
        'login' : ['./src/pages/login/index.js'],
    },
    output: {
        path: './dist',
        filename: 'js/[name].js'
    },
    externals : {
        'jquery': 'window.jQuery'
    },
    module : {
        loaders: [
            { test: /\.css$/, loader: ExtractTextPlugin.extract("style-loader","css-loader") }
        ]
    },
    plugins : [
        //独立通用模块
        new webpack.optimize.CommonsChunkPlugin({
            name : 'common',
            filename:'js/base.js'
        }),
        // 把css单独打包到文件中
        new ExtractTextPlugin("css/[name].css"),

        // html模板的处理
        new HtmlWebpackPlugin(getHtmlConfig('index')),
        new HtmlWebpackPlugin(getHtmlConfig('login'))
    ]
};

module.exports = config;

### 8. Webpack-dev-server:用于实时预览开发过程
- 首先要全局与包内安装webpack-dev-server
>npm install webpack-dev-server@1.16.5 <br>sudo npm install webpack-dev-server@1.16.5 -g
- 在entry的common模块中加入一个client
> 'common' : ['./src/page/common/index.js','webpack-dev-server/client?http://localhost:8088/']<br>
之后就可以实现自动打包了
- 实现实时预览：设置output中的publicPath(即访问url的根目录)
> 加入 PublicPath : '/dist',<br>
之后访问localhost:8088/就是访问/dist下的文件

- 开启服务：
> webpack-dev-server --inline --port 8088 <br>
运行这个命令就可以实时预览

- 分离开发环境与打包上线环境（在正式打包时忽略预览功能）
> 通过设置环境变量：var WEBPACK_ENV = process.env.WEBPACK_ENV || 'dev'; <br>
启动方式：WEBPACK_ENV=dev webpack-dev-server --inline --port 8088

为了区分线上模式与开发模式，我们设置一个判断：

In [ ]:
if('dev' === WEBPACK_ENV){
    config.entry.common.push('webpack-dev-server/client?http://localhost:8088/');

这样当WEBPACK_ENV不是dev，即开发模式时，就不会在common中加载webpack-dev-server了